# Setup

In [402]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
import statistics as stat
from IPython.display import display

class par: None
class moms: None
    
# from farhi and gourio
class true: None
true.beta = 0.961
true.mu = 1.079
true.delta = 0.02778
true.alpha = 0.244
true.g_Z = 0.01298
true.r_star = np.nan
true.g_T = np.nan
true.p = 0.034

## Settings and assumptions

In [403]:
# a. shock size
b = 0.15

# b. risk aversion coefficient
par.theta = 12

# c. IES, sigma = 1/IES
par.sigma = 0.5

In [420]:
# Tries whether this file is called from main or internally
# Ensures that we can run this code on it's own
try: 
    called_from_main
except: 
    country = "UK"
    start = '2000'
    end = '2002'
    called_from_main = 0 # if 0, print results in this notebook but don't store

# Load data

In [636]:
country = "FRA"
p2 = 0
start = 1984*(1-p2) + 2001*p2
end = 1999*(1-p2) + 2015*p2
df = pd.read_csv("data/data.csv", sep=";", index_col="year")
df = df[df["ISO"]==country]
#display(df.head())
#display(df.describe())

# Moments

In [637]:
# Start and end dates defined in main notebook
moms.ProfitK = stat.mean(df.loc[start:end,"ProfitK"])/100 
moms.ProfitY = stat.mean(df.loc[start:end,"ProfitY"])/100 
moms.XK = stat.mean(df.loc[start:end,"XK"])/100
moms.PD = stat.mean(df.loc[start:end,"PD"])
moms.TFPgrowth = stat.mean(df.loc[start:end,"TFPgrowth"])/100
moms.RF = stat.mean(df.loc[start:end,"RF"])/100 #used in step 3

for name in ['ProfitK','ProfitY','XK','PD','TFPgrowth','RF']:
    print(f'{name:10s} = {getattr(moms,name):.5f}')

ProfitK    = 0.10999
ProfitY    = 0.32407
XK         = 0.07223
PD         = 33.73328
TFPgrowth  = 0.01248
RF         = 0.04906


In [638]:
#df[["ProfitK",'ProfitY','RF','PopGrowth','PriceInvt','PD','TFPgrowth','XK','EmpPop']]

ProfitK    = 0.15348
ProfitY    = 0.35098
XK         = 0.07525
PD         = 31.77534
TFPgrowth  = 0.00787
RF         = 0.01448

# Step 1

Estimate $g_L$, $g_Q$ and $\bar{N}$ directly from data

In [639]:
par.g_L = stat.mean(df.loc[start:end,"PopGrowth"])/100
par.g_Q = -stat.mean(df.loc[start:end,"PriceInvt"])/100 # note: negativ value used
par.N_bar = stat.mean(df.loc[start:end,"EmpPop"])/100

for name in ['g_L','g_Q','N_bar']:
    print(f'{name:5s} = {getattr(par,name):.5f}')

g_L   = 0.00471
g_Q   = 0.00807
N_bar = 0.40046


# Step 2

Estimate $\mu$, $\alpha$, $\delta$, $g_z$ by solving equation system

In [640]:
def eq_footnote_15(par,moms):
     return moms.TFPgrowth - (par.g_T-(1-moms.ProfitY)*par.g_L-moms.ProfitY*(par.g_T+par.g_Q))

def eq_11(par,moms):
    return (1+par.g_T) - (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha))

def eq_15(par,moms):
    par.beta_star = 1/(1+par.r_star)
    return moms.ProfitK - ((par.mu+par.alpha-1)/par.alpha)*(par.r_star + par.delta + par.g_Q/par.beta_star) 
    
def eq_18(par,moms):
    return moms.XK - ((1+par.g_Q)*(1+par.g_T)-(1-par.delta))

def eq_20(par,moms):
    return moms.ProfitY - (par.mu+par.alpha-1)/(par.mu)

def eq_23(par,moms):
    par.beta_star = 1/(1+par.r_star)
    return moms.PD - par.beta_star*(1+par.g_T)/(1-par.beta_star*(1+par.g_T))

#def eq_27(par,moms):
#    return moms.ProfitK - (par.mu+par.alpha-1)/par.alpha*(par.r_star+par.delta+par.g_Q)

In [641]:
def set_parameters(par,x):
    
    for name,value in zip(par.names,x):
        setattr(par,name,value)
    
def set_x(par):
    
    x = np.zeros(len(par.names))
    for i,name in enumerate(par.names):
        x[i] = getattr(par,name)
        
    return x

def eq_system(x,par,moms):
    
    # a. set parameters
    set_parameters(par,x)            
    
    # c. evaluate equations
    out = []
    out.append(eq_footnote_15(par,moms))
    out.append(eq_11(par,moms))
    out.append(eq_15(par,moms))          
    out.append(eq_18(par,moms))
    out.append(eq_20(par,moms))
    out.append(eq_23(par,moms))

    return out

In [642]:
# a. parameters to estimate
par.names = ['g_Z','g_T','delta','alpha','r_star','mu']

# b. guess
par.mu = 1.01
par.delta = 0.025
par.alpha = 0.25
par.g_Z = 0.08 #0.02
par.r_star = 0.05
par.g_T = 0.04
x = set_x(par)

# c. solve
solution = optimize.fsolve(eq_system, x, args=(par,moms), full_output=0)
set_parameters(par,solution)

# d. print result
#for name in par.names:
#    print(f'{name:10} = {getattr(par,name):.8f} [FH: {getattr(true,name):.4f}]')        

# Step 3

Estimate $\beta$ and $p$

In [643]:
def update_misc(par):
    
    par.beta_star = 1/(1+par.r_star)
    par.B = np.log(1-b)
    par.Bh = np.log(1+b)
    par.g_PC = (1+par.g_T)/(1+par.g_L)-1
    

In [644]:
def find_p(p,par,moms):
        
    update_misc(par)        
    MOM2 = ((1-2*p)+p*np.exp(par.Bh*(1-par.theta)) + p*np.exp(par.B*(1-par.theta)))
    MOM3 = ((1-2*p)+p*np.exp(par.Bh*(-par.theta)) + p*np.exp(par.B*(-par.theta))) 
    
    return moms.RF - (MOM2/(par.beta_star*MOM3)-1)

In [645]:
par.p = optimize.fsolve(find_p, 0.02, args=(par,moms), full_output=0)[0]
update_misc(par)

In [646]:
MOM =  ((1-2*par.p)+par.p*np.exp(par.Bh*(1-par.theta)) + par.p*np.exp(par.B*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
par.beta = par.beta_star/((1+par.g_PC)**(-par.sigma)*MOM);

# Store or print values

In [349]:
# Collect and store results if called from main notebook
if called_from_main == 1:

    df_estimates = []
    for name in ['beta','mu','p','delta','alpha','g_L', 'g_Z', 'g_Q', 'N_bar']:
        df_estimates.append({'Name': name, str(start) + ' - ' + str(end):getattr(par,name)})
            
    df_estimates = pd.DataFrame(df_estimates)
    df_estimates = df_estimates.set_index('Name')

    # Magic - stores across notebooks
    %store df_estimates
    
# Print results if not called from main notebook
if called_from_main == 0: 
    for name in ['beta','mu','p','delta','alpha','g_L', 'g_Z', 'g_Q', 'N_bar']:
        print(f'{name:10} = {getattr(par,name):4f}')
    del called_from_main

NameError: name 'called_from_main' is not defined